In [47]:
import numpy as np
import pandas as pd
from pathlib import Path

Read both data: meteo and red days

In [48]:
DATA_FOLDER = Path("../data")

df_meteo = pd.read_feather(
    DATA_FOLDER
    / f"rtt-vent/13_most_populated_cities_data_rtt_vent_from_20170101.feather"
)
print("Shape:", df_meteo.shape)
df_meteo.head()

Shape: (65124, 15)


,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,DG,DRR,FFM,FF2M,FXY,RR
1020841,RENNES-ST JACQUES,-1.734,48.068833,20240101,5.7,13.4,9.6,9.6,7.7,0.0,868.0,5.5,NaN,10.8,11.5
1020842,RENNES-ST JACQUES,-1.734,48.068833,20240102,11.0,14.6,12.8,12.8,3.6,0.0,525.0,7.7,NaN,12.2,7.1
1020843,RENNES-ST JACQUES,-1.734,48.068833,20240103,10.4,12.8,11.2,11.6,2.4,0.0,82.0,6.9,NaN,11.4,1.4
1020844,RENNES-ST JACQUES,-1.734,48.068833,20240104,6.9,13.0,9.5,10.0,6.1,0.0,139.0,5.5,NaN,11.8,1.6
1020845,RENNES-ST JACQUES,-1.734,48.068833,20240105,3.1,11.1,6.0,7.1,8.0,0.0,140.0,2.5,NaN,6.4,2.4


In [49]:
print("Number of row with a nan value:", df_meteo.isna().sum())

Number of row with a nan value: NOM_USUEL        0
LON              0
LAT              0
AAAAMMJJ         0
TN               0
TX               0
TM              20
TNTXM            0
TAMPLI           0
DG             196
DRR          14094
FFM            118
FF2M         65124
FXY            169
RR               1
dtype: int64


In [50]:
# Print nan rows:
print("Nan rows:")
df_meteo[df_meteo.isna().any(axis=1)].head()

Nan rows:


,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,DG,DRR,FFM,FF2M,FXY,RR
1020841,RENNES-ST JACQUES,-1.734,48.068833,20240101,5.7,13.4,9.6,9.6,7.7,0.0,868.0,5.5,NaN,10.8,11.5
1020842,RENNES-ST JACQUES,-1.734,48.068833,20240102,11.0,14.6,12.8,12.8,3.6,0.0,525.0,7.7,NaN,12.2,7.1
1020843,RENNES-ST JACQUES,-1.734,48.068833,20240103,10.4,12.8,11.2,11.6,2.4,0.0,82.0,6.9,NaN,11.4,1.4
1020844,RENNES-ST JACQUES,-1.734,48.068833,20240104,6.9,13.0,9.5,10.0,6.1,0.0,139.0,5.5,NaN,11.8,1.6
1020845,RENNES-ST JACQUES,-1.734,48.068833,20240105,3.1,11.1,6.0,7.1,8.0,0.0,140.0,2.5,NaN,6.4,2.4


In [51]:
df_meteo.drop(
    ["DRR", "DG", "FF2M", "FXY", "FFM", "TM", "RR", "DRR"], axis=1, inplace=True
)

In [52]:
# Remove duplicated rows:
df_meteo.drop_duplicates(inplace=True)
print("Number of row with a nan value:", df_meteo.isna().sum().sum())

Number of row with a nan value: 0


In [53]:
df_ejp = pd.read_csv(DATA_FOLDER / "ejp_days-2016-2024.csv", sep=",")
print("Shape:", df_ejp.shape)
df_ejp.tail()

Shape: (169, 2)


,Year,Red Day Date
164,2023-2024,04/03/2024
165,2023-2024,05/03/2024
166,2023-2024,06/03/2024
167,2023-2024,07/03/2024
168,2023-2024,11/03/2024


In [54]:
df_ejp.shape

(169, 2)

Process df_ejp so that the date has the same format as in df_meteo

In [55]:
df_meteo["AAAAMMJJ"].head(), df_meteo["AAAAMMJJ"].tail()

(1020841    20240101
 1020842    20240102
 1020843    20240103
 1020844    20240104
 1020845    20240105
 Name: AAAAMMJJ, dtype: int64,
 12747951    20241229
 12747952    20241230
 12747953    20241231
 12747954    20250101
 12747955    20250102
 Name: AAAAMMJJ, dtype: int64)

In [56]:
df_ejp["formatted_red_day_date"] = pd.to_datetime(
    df_ejp["Red Day Date"], format="%d/%m/%Y"
).dt.strftime("%Y%m%d")

df_ejp["formatted_red_day_date"] = df_ejp["formatted_red_day_date"].astype(int)
df_ejp["formatted_red_day_date"].head()

0    20161220
1    20170103
2    20170104
3    20170105
4    20170106
Name: formatted_red_day_date, dtype: int32

In [57]:
# remove 2016 value
df_ejp = df_ejp[df_ejp["formatted_red_day_date"] >= 20170101]
df_ejp.tail()

,Year,Red Day Date,formatted_red_day_date
164,2023-2024,04/03/2024,20240304
165,2023-2024,05/03/2024,20240305
166,2023-2024,06/03/2024,20240306
167,2023-2024,07/03/2024,20240307
168,2023-2024,11/03/2024,20240311


In [58]:
# Create a new dataframe with the date from 20170101 to 20240311 and whheter or not it was a red day
date_range = pd.date_range(start="2017-01-01", end="2024-03-11")
df_red_days = pd.DataFrame({"AAAAMMJJ": date_range.strftime("%Y%m%d").astype(int)})

df_red_days["is_red_day"] = df_red_days["AAAAMMJJ"].isin(
    df_ejp["formatted_red_day_date"]
)

print("Shape:", df_red_days.shape)
df_red_days.head()

Shape: (2627, 2)


,AAAAMMJJ,is_red_day
0,20170101,False
1,20170102,False
2,20170103,True
3,20170104,True
4,20170105,True


In [59]:
df_red_days["is_red_day"].value_counts()

is_red_day
False    2459
True      168
Name: count, dtype: int64

Further process the meteo data so that the format is ready for trainings

In [60]:
# Sort by date and remove data prior to 2017
df_meteo = df_meteo.sort_values("AAAAMMJJ")
df_meteo = df_meteo[df_meteo["AAAAMMJJ"] >= 20170101]

df_meteo = df_meteo.drop_duplicates(keep="first")

df_meteo.head()

,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TNTXM,TAMPLI
6197286,TOULOUSE-BLAGNAC,1.378833,43.621000,20170101,-2.1,2.7,0.3,4.8
3610371,MARSEILLE,5.479167,43.310667,20170101,4.5,13.9,9.2,9.4
8621740,REIMS-PRUNAY,4.155333,49.209833,20170101,-4.4,-0.2,-2.3,4.2
6662062,MONTPELLIER-AEROPORT,3.964667,43.576167,20170101,2.4,12.3,7.4,9.9
7848143,NANTES-BOUGUENAIS,-1.608833,47.150000,20170101,-4.0,5.2,0.6,9.2


In [61]:
# Process NOM_USUEL col to remove values after '-' or '_':
df_meteo["NOM_USUEL"] = df_meteo["NOM_USUEL"].str.split("_|-").str[0]

# Remove LON and LAT cols
df_meteo.drop(["LON", "LAT"], axis=1, inplace=True)

df_meteo.head()

,NOM_USUEL,AAAAMMJJ,TN,TX,TNTXM,TAMPLI
6197286,TOULOUSE,20170101,-2.1,2.7,0.3,4.8
3610371,MARSEILLE,20170101,4.5,13.9,9.2,9.4
8621740,REIMS,20170101,-4.4,-0.2,-2.3,4.2
6662062,MONTPELLIER,20170101,2.4,12.3,7.4,9.9
7848143,NANTES,20170101,-4.0,5.2,0.6,9.2


In [62]:
# Pivot so that each row is a day and each column is a measure per station
pivoted_df_meteo = df_meteo.pivot(index="AAAAMMJJ", columns="NOM_USUEL")

# Flatten MultiIndex columns
pivoted_df_meteo.columns = [
    "{}_{}".format(col[0], col[1]) for col in pivoted_df_meteo.columns
]

pivoted_df_meteo.reset_index(inplace=True)
print("Shape:", pivoted_df_meteo.shape)
pivoted_df_meteo.head()

Shape: (2989, 53)


,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_REIMS,...,TAMPLI_MARSEILLE,TAMPLI_MONTPELLIER,TAMPLI_NANTES,TAMPLI_NICE,TAMPLI_PARIS,TAMPLI_REIMS,TAMPLI_RENNES,TAMPLI_STRASBOURG,TAMPLI_TOULON,TAMPLI_TOULOUSE
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-4.4,...,9.4,9.9,9.2,10.3,5.7,4.2,7.3,2.2,10.5,4.8
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-2.1,...,7.9,9.0,2.0,10.5,3.4,2.9,3.6,1.8,6.6,1.9
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-1.0,...,11.6,11.3,3.8,9.7,2.4,3.2,4.5,3.7,10.2,2.1
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,0.5,...,10.9,14.4,9.3,7.8,6.5,4.8,9.3,5.0,11.1,3.8
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,0.4,...,5.9,8.5,7.6,8.6,5.1,4.4,7.2,4.7,7.1,7.9


Merge dataframes

In [63]:
df_merged = pd.merge(pivoted_df_meteo, df_red_days, on="AAAAMMJJ", how="left")
df_merged.head()

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_REIMS,...,TAMPLI_MONTPELLIER,TAMPLI_NANTES,TAMPLI_NICE,TAMPLI_PARIS,TAMPLI_REIMS,TAMPLI_RENNES,TAMPLI_STRASBOURG,TAMPLI_TOULON,TAMPLI_TOULOUSE,is_red_day
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-4.4,...,9.9,9.2,10.3,5.7,4.2,7.3,2.2,10.5,4.8,False
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-2.1,...,9.0,2.0,10.5,3.4,2.9,3.6,1.8,6.6,1.9,False
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-1.0,...,11.3,3.8,9.7,2.4,3.2,4.5,3.7,10.2,2.1,True
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,0.5,...,14.4,9.3,7.8,6.5,4.8,9.3,5.0,11.1,3.8,True
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,0.4,...,8.5,7.6,8.6,5.1,4.4,7.2,4.7,7.1,7.9,True


In [64]:
# Remove data prior to 2024-04
df_merged = df_merged[df_merged["AAAAMMJJ"] <= 20240311]

In [65]:
# Show number of nans:
print("Number of row with a nan value:", df_merged.isna().sum().sum())

Number of row with a nan value: 1460


In [66]:
# show nan rows:
print("Nan rows:")
df_merged[df_merged.isna().any(axis=1)].head()

Nan rows:


,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_REIMS,...,TAMPLI_MONTPELLIER,TAMPLI_NANTES,TAMPLI_NICE,TAMPLI_PARIS,TAMPLI_REIMS,TAMPLI_RENNES,TAMPLI_STRASBOURG,TAMPLI_TOULON,TAMPLI_TOULOUSE,is_red_day
2191,20230101,9.6,10.6,13.4,13.0,NaN,11.9,10.9,11.4,10.4,...,NaN,2.9,4.8,3.1,5.0,2.3,7.5,1.5,3.9,False
2192,20230102,9.4,8.1,10.3,11.5,NaN,8.9,10.5,9.2,5.2,...,NaN,5.2,6.1,2.7,6.7,6.2,8.1,2.7,2.6,False
2193,20230103,3.4,2.8,7.3,9.1,NaN,5.1,11.6,5.4,1.3,...,NaN,8.3,5.7,5.4,9.2,7.6,9.8,6.6,5.2,False
2194,20230104,3.7,8.7,1.2,5.1,NaN,11.2,10.4,9.2,7.4,...,NaN,2.7,6.3,4.4,6.3,3.2,10.9,10.5,7.6,False
2195,20230105,10.0,9.4,8.0,6.7,NaN,11.7,9.1,11.8,10.5,...,NaN,1.9,6.7,2.1,3.6,2.3,4.0,12.8,9.8,False


In [67]:
# print number of nans by column:
print("Number of nans by column:")
pd.set_option("display.max_rows", None)
df_merged.isna().sum()

Number of nans by column:


AAAAMMJJ                0
TN_BORDEAUX             0
TN_LILLE                0
TN_LYON                 0
TN_MARSEILLE            0
TN_MONTPELLIER        365
TN_NANTES               0
TN_NICE                 0
TN_PARIS                0
TN_REIMS                0
TN_RENNES               0
TN_STRASBOURG           0
TN_TOULON               0
TN_TOULOUSE             0
TX_BORDEAUX             0
TX_LILLE                0
TX_LYON                 0
TX_MARSEILLE            0
TX_MONTPELLIER        365
TX_NANTES               0
TX_NICE                 0
TX_PARIS                0
TX_REIMS                0
TX_RENNES               0
TX_STRASBOURG           0
TX_TOULON               0
TX_TOULOUSE             0
TNTXM_BORDEAUX          0
TNTXM_LILLE             0
TNTXM_LYON              0
TNTXM_MARSEILLE         0
TNTXM_MONTPELLIER     365
TNTXM_NANTES            0
TNTXM_NICE              0
TNTXM_PARIS             0
TNTXM_REIMS             0
TNTXM_RENNES            0
TNTXM_STRASBOURG        0
TNTXM_TOULON

In [68]:
df_merged.to_feather(DATA_FOLDER / "merged_meteo_red_days_from_20170101.feather")